In [1]:
# Here is the code without the use of a Flask server.

# packages
from textblob import TextBlob
import xml.etree.ElementTree as ET
import requests
import random

base_url = 'http://export.arxiv.org/api/query?'

# function for /get_data
def fetch_articles():
  start_index = random.randint(0, 1000)
  search_query = f'search_query=all:AI&start={start_index}&max_results=5'
  query_url = f"{base_url}{search_query}"
  response = requests.get(query_url)
  articles = []

  if response.status_code == 200:
    root = ET.fromstring(response.content)
    ns = {'atom': 'http://www.w3.org/2005/Atom'}

    for entry in root.findall('atom:entry', ns): #we recover the desired data
      title = entry.find('atom:title', ns).text.strip()
      summary = entry.find('atom:summary', ns).text.strip()
      published = entry.find('atom:published', ns).text
      authors = [author.find('atom:name', ns).text for author in entry.findall('atom:author', ns)]

      #we consider : get full content of the article = recover the pdf link
      pdf_link = None
      for link in entry.findall('atom:link', ns):
        if 'title' in link.attrib and link.attrib['title'] == 'pdf':
          pdf_link = link.attrib['href']
          break

      articles.append({'title': title,'summary': summary[:150] + "...",'published': published,'authors': ', '.join(authors),'pdf_link': pdf_link  })
  else:
    print(f"Oups we failed to fetch articles")

  return articles

#functions for /articles
def print_get_data(articles):
  for i, article in enumerate(articles, start=1):
    print(f"Article #{i}: {article['title']}")

#function for /article/<number>
def print_articles_details(articles):
  for i, article in enumerate(articles, start=1):
    print(f"Article #{i}:")
    print(f"Title: {article['title']}")
    print(f"Published: {article['published']}")
    print(f"Authors: {article['authors']}")
    print(f"Summary: {article['summary']}\n")

#function for /article/<number>
def print_article_content(articles,number):
  if 1 <= number <= len(articles):
    article = articles[number - 1]
    print(f"Article #{number} content:")
    print(f"Title: {article['title']}")
    print(f"Published: {article['published']}")
    print(f"Authors: {article['authors']}")
    print(f"Summary: {article['summary']}")
    print(f"PDF Link: {article['pdf_link']}\n")
  else:
    print(f"Out of range. Please choose a number between 1 and {len(articles)}.")

#function for /ml
def perform_sentiment_analysis(content):
  analysis = TextBlob(content)
  return analysis.sentiment

def print_sentiment_analysis(articles):
  for i, article in enumerate(articles, start=1): #all articles
    sentiment = perform_sentiment_analysis(article['summary'])
    print(f"Sentiment analysis of Article #{i}: {sentiment}")

#fetch articles one time
articles = fetch_articles()

#Execution
print("Fetching data...\n")
print_get_data(articles)
print("\nFetching detailed articles information...\n")
print_articles_details(articles)
print("\nFetching content of a specific article...\n")
numero = int(input("Please choose an article number between 1 and 5: "))
print_article_content(articles, numero)
print("\nPerforming sentiment analysis on all articles...\n")
print_sentiment_analysis(articles)

Fetching data...

Article #1: Causal Learning for Socially Responsible AI
Article #2: Bridging the Gap Between Explainable AI and Uncertainty Quantification
  to Enhance Trustability
Article #3: A Psychopathological Approach to Safety Engineering in AI and AGI
Article #4: An Explorative Study of GitHub Repositories of AI Papers
Article #5: Better Future through AI: Avoiding Pitfalls and Guiding AI Towards its
  Full Potential

Fetching detailed articles information...

Article #1:
Title: Causal Learning for Socially Responsible AI
Published: 2021-04-25T22:09:11Z
Authors: Lu Cheng, Ahmadreza Mosallanezhad, Paras Sheth, Huan Liu
Summary: There have been increasing concerns about Artificial Intelligence (AI) due to
its unfathomable potential power. To make AI address ethical challenges ...

Article #2:
Title: Bridging the Gap Between Explainable AI and Uncertainty Quantification
  to Enhance Trustability
Published: 2021-05-25T10:53:58Z
Authors: Dominik Seuß
Summary: After the tremendous a